# Preprocessing Train Set

In [202]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

# Load dataset
train = pd.read_csv("C:/Users/fabri/Desktop/uni/MSDS/2024-2025/Semester 2/Advanced Analytics in a Big Data World/Project/Assignment 1/Data/train.csv")

# Drop duplicate rows while ignoring the "id" column
train = train.drop_duplicates(subset=[col for col in train.columns if col != "id"])

In [203]:
# Define mapping for dwelling categories
dwelling_map = {
    **{dw: "Apartments" for dw in [
        "Appartement", "Appartementengebouw", "Assistentie-appartement", "Bel-Étage", "Dakappartement",
        "Duplex", "Gelijkvloers app.", "Loft", "Penthouse", "Serviceflat", "Studio", "Studio met slaaphoek"
    ]},
    **{dw: "Houses" for dw in [
        "Arbeiderswoning", "Boerderij", "Bungalow", "Burgerswoning", "Cottage", "Eengezinswoning",
        "Fermette", "Herenhuis", "Herenwoning", "Hoekwoning", "Hoeve", "Kangoeroewoning", "Koppelvilla",
        "Koppelwoning", "Moderne villa", "Pastorijwoning", "Rijwoning", "Villa", "Villa-landhuis", "Woning"
    ]},
    **{dw: "Other" for dw in [
        "Andere", "Buitenverblijf", "Gemengd gebruik", "Kasteel", "Uitzonderlijke woning",
        "Vakantiewoning", "Woonboot"
    ]}
}

# Categorize dwellings using a fast dictionary lookup
train["Category"] = train["subtype"].map(dwelling_map).fillna("Other")


In [204]:
# Impute missing numerical values using the median within each category
imputed_cols = ["area", "energy_value"]
train[imputed_cols] = train.groupby("Category")[imputed_cols].transform(lambda x: x.fillna(x.median()))

In [205]:
# Handle postcodes
def classify_location(postcode):
    if pd.isna(postcode):
        return "Unknown"  # Handle missing postcodes
    
    postcode = str(postcode).zfill(4)  # Ensure it's a 4-digit string
    first_digit, second_digit, third_digit = map(int, postcode[:3])  # Convert only once
    
    if first_digit in {1, 2, 3, 4, 9} and second_digit in {0, 1, 2, 3, 4} and third_digit in {0, 1}:
        return "Urban"
    elif (
        (first_digit in {1, 2, 3, 4, 9} and second_digit in {5, 6, 7}) or
        (first_digit in {5, 6, 7, 8} and second_digit in {0, 1, 2, 3, 4}) or
        (first_digit in {5, 6, 7, 8} and second_digit in {5, 6} and third_digit in {0, 1, 2, 3, 4, 5})
    ):
        return "Suburban"
    else:
        return "Rural"

# Apply classification to postcodes
train["Location_Type"] = train["postcode"].apply(classify_location)
train = pd.get_dummies(train, columns=["Location_Type"], drop_first=True)


In [206]:
# Define energy label bins and corresponding labels
energy_bins = np.array([-float("inf"), 100, 200, 300, 400, 500, float("inf")])
energy_labels = np.array(["A", "B", "C", "D", "E", "F"])

# Fast classification function using numpy search
def classify_energy_label(energy):
    if pd.isna(energy):
        return "Unknown"  # Handle missing values
    return energy_labels[np.searchsorted(energy_bins, energy, side="right") - 1]

# Apply classification
train["energy_label"] = train["energy_value"].apply(classify_energy_label)

# One-hot encode energy labels
train = pd.get_dummies(train, columns=["energy_label"], prefix="energy")

In [207]:
# Encode "Category" as an ordinal variable
train["Category"] = train["Category"].astype("category").cat.codes

In [208]:
# Encoding of "province"
if train["province"].nunique() > 10:  # If high cardinality, use ordinal encoding
    train["province"] = train["province"].astype("category").cat.codes
else:  # If low cardinality, use one-hot encoding
    train = pd.get_dummies(train, columns=["province"], drop_first=True)

# Convert "province" to float
train["province"].fillna(train["province"].median(), inplace=True)  # If missing
train["province"] = train["province"].astype(int)  # Convert to integer


C:\Users\fabri\AppData\Local\Temp\ipykernel_19472\1816923625.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train["province"].fillna(train["province"].median(), inplace=True)  # If missing


In [209]:
# Use KNN imputation for missing values in "lat" and "lon"
imputer = KNNImputer(n_neighbors=5)
train[["lat", "lon"]] = imputer.fit_transform(train[["lat", "lon"]])

In [210]:
# Normalize features
scaler = MinMaxScaler()
continuous_cols = ["area", "energy_value", "foto_amount", "price"]
original_prices = train["price"].copy() # Save original prices before scaling
scaler.fit(train[continuous_cols])
train[continuous_cols] = scaler.transform(train[continuous_cols])
train["price"] = scaler.inverse_transform(train[continuous_cols])[:, -1]  # Inverse transform price

print("Original Price Sample:\n", original_prices.head())
print("Restored Price Sample:\n", train["price"].head())

Original Price Sample:
 0    599000.0
1    199000.0
2    875000.0
3    649000.0
4    765000.0
Name: price, dtype: float64
Restored Price Sample:
 0    599000.0
1    199000.0
2    875000.0
3    649000.0
4    765000.0
Name: price, dtype: float64


In [211]:
# Ensure all column names are clean for modeling
train.columns = train.columns.str.replace(" ", "_").str.lower()

# Convert Boolean columns to integers
bool_cols = train.select_dtypes(include=['bool']).columns
train[bool_cols] = train[bool_cols].astype(int)


# Drop columns
dropped = ['id', 'is_appartment', 'postcode', 'added_time', 'advertiser', 'is_promoted', 'sticker', 'price_drop_date', 'subtype']
train.drop(dropped, axis=1, inplace=True)

In [212]:
# Save preprocessed data
train.to_csv("C:/Users/fabri/Desktop/uni/MSDS/2024-2025/Semester 2/Advanced Analytics in a Big Data World/Project/Assignment 1/Data/train_preprocessed.csv", index=False)

# Preprocessing Test Set

In [213]:
# Load dataset
test = pd.read_csv("C:/Users/fabri/Desktop/uni/MSDS/2024-2025/Semester 2/Advanced Analytics in a Big Data World/Project/Assignment 1/Data/test.csv")

# Drop duplicate rows while ignoring the "id" column
test = test.drop_duplicates(subset=[col for col in test.columns if col != "id"])

In [214]:
# Define mapping for dwelling categories
dwelling_map = {
    **{dw: "Apartments" for dw in [
        "Appartement", "Appartementengebouw", "Assistentie-appartement", "Bel-Étage", "Dakappartement",
        "Duplex", "Gelijkvloers app.", "Loft", "Penthouse", "Serviceflat", "Studio", "Studio met slaaphoek"
    ]},
    **{dw: "Houses" for dw in [
        "Arbeiderswoning", "Boerderij", "Bungalow", "Burgerswoning", "Cottage", "Eengezinswoning",
        "Fermette", "Herenhuis", "Herenwoning", "Hoekwoning", "Hoeve", "Kangoeroewoning", "Koppelvilla",
        "Koppelwoning", "Moderne villa", "Pastorijwoning", "Rijwoning", "Villa", "Villa-landhuis", "Woning"
    ]},
    **{dw: "Other" for dw in [
        "Andere", "Buitenverblijf", "Gemengd gebruik", "Kasteel", "Uitzonderlijke woning",
        "Vakantiewoning", "Woonboot"
    ]}
}

# Categorize dwellings using a fast dictionary lookup
test["Category"] = test["subtype"].map(dwelling_map).fillna("Other")

In [215]:
# Impute missing numerical values using the median within each category
imputed_cols = ["area", "energy_value"]
test[imputed_cols] = test.groupby("Category")[imputed_cols].transform(lambda x: x.fillna(x.median()))

In [216]:
# Postcode handling
def classify_location(postcode):
    if pd.isna(postcode):
        return "Unknown"  # Handle missing postcodes
    
    postcode = str(postcode).zfill(4)  # Ensure it's a 4-digit string
    first_digit, second_digit, third_digit = map(int, postcode[:3])  # Convert only once
    
    if first_digit in {1, 2, 3, 4, 9} and second_digit in {0, 1, 2, 3, 4} and third_digit in {0, 1}:
        return "Urban"
    elif (
        (first_digit in {1, 2, 3, 4, 9} and second_digit in {5, 6, 7}) or
        (first_digit in {5, 6, 7, 8} and second_digit in {0, 1, 2, 3, 4}) or
        (first_digit in {5, 6, 7, 8} and second_digit in {5, 6} and third_digit in {0, 1, 2, 3, 4, 5})
    ):
        return "Suburban"
    else:
        return "Rural"

# Apply classification to postcodes
test["Location_Type"] = test["postcode"].apply(classify_location)
test = pd.get_dummies(test, columns=["Location_Type"], drop_first=True)

In [217]:
# Define energy label bins and corresponding labels
energy_bins = np.array([-float("inf"), 100, 200, 300, 400, 500, float("inf")])
energy_labels = np.array(["A", "B", "C", "D", "E", "F"])

# Fast classification function using numpy search
def classify_energy_label(energy):
    if pd.isna(energy):
        return "Unknown"  # Handle missing values
    return energy_labels[np.searchsorted(energy_bins, energy, side="right") - 1]

# Apply classification
test["energy_label"] = test["energy_value"].apply(classify_energy_label)

# One-hot encode energy labels
test = pd.get_dummies(test, columns=["energy_label"], prefix="energy")

In [218]:
# Encode "Category" as an ordinal variable
test["Category"] = test["Category"].astype("category").cat.codes

In [219]:
# Encoding of "province"
if test["province"].nunique() > 10:  # If high cardinality, use ordinal encoding
    test["province"] = test["province"].astype("category").cat.codes
else:  # If low cardinality, use one-hot encoding
    test = pd.get_dummies(test, columns=["province"], drop_first=True)

# Convert "province" to float
test["province"].fillna(test["province"].median(), inplace=True)  # If missing
test["province"] = test["province"].astype(int)  # Convert to integer


C:\Users\fabri\AppData\Local\Temp\ipykernel_19472\118521211.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test["province"].fillna(test["province"].median(), inplace=True)  # If missing


In [220]:
# Use KNN imputation for missing values in "lat" and "lon"
imputer = KNNImputer(n_neighbors=5)
test[["lat", "lon"]] = imputer.fit_transform(test[["lat", "lon"]])

In [221]:
# Normalize features
scaler = MinMaxScaler()
continuous_cols = ["area", "energy_value", "foto_amount"]
scaler.fit(train[continuous_cols])
test[continuous_cols] = scaler.transform(test[continuous_cols])

In [222]:
# Ensure all column names are clean for modeling
test.columns = test.columns.str.replace(" ", "_").str.lower()

# Convert Boolean columns to integers
bool_cols = test.select_dtypes(include=['bool']).columns
test[bool_cols] = test[bool_cols].astype(int)


# Drop columns
dropped = ['is_appartment', 'postcode', 'added_time', 'advertiser', 'is_promoted', 'sticker', 'price_drop_date', 'subtype']
test.drop(dropped, axis=1, inplace=True)

In [223]:
# Save preprocessed data
test.to_csv("C:/Users/fabri/Desktop/uni/MSDS/2024-2025/Semester 2/Advanced Analytics in a Big Data World/Project/Assignment 1/Data/test_preprocessed.csv", index=False)

# Train XGBoost Model

In [179]:
import optuna
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error

In [188]:
# Load preprocessed data
train = pd.read_csv("C:/Users/fabri/Desktop/uni/MSDS/2024-2025/Semester 2/Advanced Analytics in a Big Data World/Project/Assignment 1/Data/train_preprocessed.csv")
test = pd.read_csv("C:/Users/fabri/Desktop/uni/MSDS/2024-2025/Semester 2/Advanced Analytics in a Big Data World/Project/Assignment 1/Data/test_preprocessed.csv")
test_original = pd.read_csv("C:/Users/fabri/Desktop/uni/MSDS/2024-2025/Semester 2/Advanced Analytics in a Big Data World/Project/Assignment 1/Data/test.csv")

# Define target and features
X = train.drop("price", axis=1)
y = train["price"]

# Split data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

# Drop irrelevant columns (if they are not needed for prediction)
drop_cols = ["id", "advertiser", "sticker", "price_drop_date", "subtype"]
train = train.drop(columns=[col for col in drop_cols if col in train.columns], errors="ignore")

# Convert 'province' to numeric if it's categorical
if train["province"].dtype == "object":
    train["province"] = train["province"].astype("category").cat.codes

# Ensure all columns are numeric
print(train.dtypes)  # Check again to confirm all are int/float

area                      float64
bedrooms                  float64
new_building                int64
lat                       float64
lon                       float64
foto_amount               float64
energy_value              float64
province                    int64
price                     float64
category                    int64
location_type_suburban      int64
location_type_urban         int64
energy_a                    int64
energy_b                    int64
energy_c                    int64
energy_d                    int64
energy_e                    int64
energy_f                    int64
dtype: object


In [182]:
# Explicitly mark categorical features
categorical_cols = [
    "new_building", "province", "category", "location_type_suburban", "location_type_urban",
    "energy_a", "energy_b", "energy_c", "energy_d", "energy_e", "energy_f"
]
for col in categorical_cols:
    X[col] = X[col].astype("category").cat.codes
X = X.astype("float32") # Convert to float32 for XGBoost
print(X.dtypes)

area                      float32
bedrooms                  float32
new_building              float32
lat                       float32
lon                       float32
foto_amount               float32
energy_value              float32
province                  float32
category                  float32
location_type_suburban    float32
location_type_urban       float32
energy_a                  float32
energy_b                  float32
energy_c                  float32
energy_d                  float32
energy_e                  float32
energy_f                  float32
dtype: object


In [183]:
# Define Optuna objective function (Using Validation Set)
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=50),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-3, 10, log=True),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
    }

 # Train model on Train set, evaluate on Validation set
    model = xgb.XGBRegressor(**params, random_state=42)
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        verbose=False
    )

    # Evaluate using the Validation set
    y_valid_pred = model.predict(X_valid)
    return mean_absolute_error(y_valid, y_valid_pred)

In [184]:
# Run Optuna Optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100, timeout=1200)

# Get best hyperparameters
best_params = study.best_params
print("Best parameters:", best_params)

[I 2025-03-13 22:46:59,387] A new study created in memory with name: no-name-492187b0-a10b-413b-9798-184204f12677
[I 2025-03-13 22:46:59,849] Trial 0 finished with value: 72466.41160212002 and parameters: {'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.2198572525894275, 'subsample': 0.8667755794523004, 'colsample_bytree': 0.8279238629699699, 'reg_lambda': 2.018397162004448, 'reg_alpha': 0.04471608669032889, 'min_child_weight': 2}. Best is trial 0 with value: 72466.41160212002.
[I 2025-03-13 22:47:01,542] Trial 1 finished with value: 79258.1803096038 and parameters: {'n_estimators': 400, 'max_depth': 4, 'learning_rate': 0.016958915528453213, 'subsample': 0.7006442143118643, 'colsample_bytree': 0.5835184791862003, 'reg_lambda': 0.23023094323718427, 'reg_alpha': 0.032893113458393165, 'min_child_weight': 7}. Best is trial 0 with value: 72466.41160212002.
[I 2025-03-13 22:47:09,203] Trial 2 finished with value: 70192.19326723239 and parameters: {'n_estimators': 950, 'max_depth': 9,

Best parameters: {'n_estimators': 600, 'max_depth': 11, 'learning_rate': 0.03253573886681746, 'subsample': 0.9159706162703845, 'colsample_bytree': 0.8629834019917649, 'reg_lambda': 0.003038522321115552, 'reg_alpha': 0.5600114325184049, 'min_child_weight': 1}


In [185]:
# Train model on full train set
best_model = xgb.XGBRegressor(**best_params, random_state=42)
best_model.fit(X, y)

# MAE on validation set
y_valid_pred = best_model.predict(X_valid)
mae_valid = mean_absolute_error(y_valid, y_valid_pred)
print("Validation MAE:", mae_valid)

# Predict on test set
test_pred = best_model.predict(test)
print(test_pred[:10])

Validation MAE: 15228.954789909638
[648911.94 625524.8  771511.1  671364.9  677963.6  796948.44 637025.8
 817065.2  761074.56 762620.9 ]


In [198]:
# Uncertainty bounds
lower_bound = test_pred * 0.90 # 90% of predicted price
upper_bound = test_pred * 1.10 # 110% of predicted price
lower_bound = lower_bound.round(0) # Round to nearest integer
upper_bound = upper_bound.round(0) # Round to nearest integer
test_pred = test_pred.round(0) # Round to nearest integer

# Create submission
submission = pd.DataFrame({
    "ID": test_original["id"].astype(str),
    "LOWER": lower_bound,
    "UPPER": upper_bound,
    "PRED": test_pred
})
submission.to_csv("C:/Users/fabri/Desktop/uni/MSDS/2024-2025/Semester 2/Advanced Analytics in a Big Data World/Project/Assignment 1/Data/submission.csv", index=False)
print("Submission saved to submission.csv")
print(submission.head())

Submission saved to submission.csv
    ID     LOWER     UPPER      PRED
0  te0  584021.0  713803.0  648912.0
1  te1  562972.0  688078.0  625525.0
2  te2  694360.0  848662.0  771511.0
3  te3  604228.0  738502.0  671365.0
4  te4  610168.0  745760.0  677964.0
